## Introducción a AWS KMS

AWS Key Management Service (AWS KMS) es un servicio administrado que le facilita la creación y el control de las claves criptográficas que se utilizan para proteger sus datos. AWS KMS utiliza módulos de seguridad de hardware (HSM) para proteger y validar sus claves de AWS KMS conforme al programa de validación de módulos criptográficos FIPS 140-2, excepto en las regiones de China (Pekín) y China (Ningxia).

Al utilizar AWS KMS, obtiene más control sobre el acceso a los datos que cifra. Puede utilizar las características criptográficas y de administración de claves directamente en sus aplicaciones o a través de los servicios de AWS integrados con AWS KMS. Tanto si escribe aplicaciones para AWS como si utiliza servicios de AWS, AWS KMS le permite mantener el control sobre quién puede utilizar sus claves de AWS KMS y obtener acceso a sus datos cifrados.

AWS KMS se integra con AWS CloudTrail, un servicio que entrega archivos de registro a su bucket de Amazon S3 designado. Mediante CloudTrail puede monitorizar e investigar cómo y cuándo se han utilizado sus claves KMS y quién las ha utilizado.

In [7]:
import boto3
import json
import base64

In [6]:
# Create session
session = boto3.Session(profile_name='badbob')
kms_client = session.client('kms')

In [9]:
# Create key
 
response = kms_client.create_key(
    Description='My KMS key',
    KeyUsage='ENCRYPT_DECRYPT',
    Origin='AWS_KMS'
)

key_id = response['KeyMetadata']['KeyId']
    

Los alias de las llaves son muy útiles. Son más fáciles de recordar cuando se opera con claves. Y lo que es más importante, al rotar claves, como veremos más adelante en esta sección, no tendremos que actualizar nuestro código para actualizar los nuevos KeyIDs o referencias ARN. Usando alias en nuestro código para llamar a las CMKs por ellos, y actualizando los alias CMKs para que apunten a la nueva clave generada, la cantidad de cambios en nuestro código se minimiza.

In [11]:
# Create alias for key
kms_client.create_alias(
    AliasName='alias/MyKeyAlias',
    TargetKeyId=key_id
)

{'ResponseMetadata': {'RequestId': '59d60b0f-6fd1-47f8-8be4-0aaba6e57e25',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '59d60b0f-6fd1-47f8-8be4-0aaba6e57e25',
   'cache-control': 'no-cache, no-store, must-revalidate, private',
   'expires': '0',
   'pragma': 'no-cache',
   'date': 'Fri, 28 Jul 2023 14:26:39 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

La **rotación de claves** es muy importante en la administración de claves y una práctica recomendada de seguridad. En AWS KMS existen diferentes formas de rotar las claves en función de cómo se crearon.

En el caso de las CMK creadas con material de clave de AWS, AWS KMS también guarda el material criptográfico más antiguo de la CMK, por lo que se puede utilizar para descifrar los datos que cifró. La rotación automática de claves conserva las propiedades de la CMK: el ID de la clave, el ARN de la clave, la región, las políticas y los permisos no cambian cuando se rota la clave, por lo que no es necesario actualizar manualmente el alias de la CMK para que apunte a una CMK recién generada.

In [13]:
# Rotate key every 30 days
kms_client.enable_key_rotation(
    KeyId=key_id
)  

{'ResponseMetadata': {'RequestId': '16f71b4a-e8f3-4f3a-be79-a42528b36d69',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '16f71b4a-e8f3-4f3a-be79-a42528b36d69',
   'cache-control': 'no-cache, no-store, must-revalidate, private',
   'expires': '0',
   'pragma': 'no-cache',
   'date': 'Fri, 28 Jul 2023 14:31:45 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

Otra forma de rotar las CMKs construidas con material de llave AWS es generar una nueva llave, luego actualizar el Alias para la CMK.

In [14]:
# Create key
 
response = kms_client.create_key(
    Description='My new KMS key',
    KeyUsage='ENCRYPT_DECRYPT',
    Origin='AWS_KMS'
)

key_id_new = response['KeyMetadata']['KeyId']

In [15]:
# Update alias
kms_client.update_alias(
    AliasName='alias/MyKeyAlias',
    TargetKeyId=key_id_new
)

{'ResponseMetadata': {'RequestId': 'b0ffff69-a40b-4857-b326-aed9aba10e31',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b0ffff69-a40b-4857-b326-aed9aba10e31',
   'cache-control': 'no-cache, no-store, must-revalidate, private',
   'expires': '0',
   'pragma': 'no-cache',
   'date': 'Fri, 28 Jul 2023 14:36:24 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

La eliminación de claves maestras de cliente es una operación muy delicada. Sólo debes borrar una CMK cuando estés seguro de que ya no necesitas utilizarla.

In [16]:
kms_client.disable_key(
    KeyId=key_id
)

{'ResponseMetadata': {'RequestId': '8d8e34b3-7e5a-41fe-84f0-1857a96bb896',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8d8e34b3-7e5a-41fe-84f0-1857a96bb896',
   'cache-control': 'no-cache, no-store, must-revalidate, private',
   'expires': '0',
   'pragma': 'no-cache',
   'date': 'Fri, 28 Jul 2023 14:38:55 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

Para la operación de eliminación, AWS KMS impone un periodo de espera. Para eliminar una CMK en AWS KMS tiene que programar la eliminación de una clave. Puede establecer el periodo de espera desde un mínimo de 7 días hasta un máximo de 30 días (por defecto).

In [17]:
kms_client.schedule_key_deletion(
    KeyId=key_id,
    PendingWindowInDays=7
)

{'KeyId': 'arn:aws:kms:us-east-1:535781350567:key/7de9c751-e0e6-4703-9719-51e67bdca218',
 'DeletionDate': datetime.datetime(2023, 8, 4, 11, 45, 2, 233000, tzinfo=tzlocal()),
 'KeyState': 'PendingDeletion',
 'PendingWindowInDays': 7,
 'ResponseMetadata': {'RequestId': '31bbd22a-9bdc-44f6-b641-ebdee29f43bd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '31bbd22a-9bdc-44f6-b641-ebdee29f43bd',
   'cache-control': 'no-cache, no-store, must-revalidate, private',
   'expires': '0',
   'pragma': 'no-cache',
   'date': 'Fri, 28 Jul 2023 14:45:02 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '172',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}